<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/044_ToolDecorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧰 Tool Decorators: Keeping Code and Documentation in Sync

When building agents, one challenge is keeping the **tool implementation** (i.e., Python functions) in sync with their **metadata** (descriptions, parameter schemas, etc). When done manually, it’s easy for descriptions or parameters to become outdated.

### 🔥 Problem Without Decorators

Traditionally, you manually register a tool like this:

```python
# Registering the tool manually
action_registry.register(Action(
    name="read_file",
    function=read_file,
    description="Reads content from a specified file",
    parameters={
        "type": "object",
        "properties": {
            "file_path": {"type": "string"}
        },
        "required": ["file_path"]
    }
))

# The actual function
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file."""
    with open(file_path, 'r') as f:
        return f.read()
```

**Downside:** Every time you update the function, you must also update the schema and description separately. Easy to forget!

---

## ✅ Solution: Use Python Decorators

Instead, use a decorator that automatically extracts the tool name, docstring, and parameter types from the function itself.

### 🧪 Example

```python
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    """
    Reads and returns the content of a file from the specified path.

    Args:
        file_path: The path to the file to read

    Returns:
        The contents of the file as a string
    """
    with open(file_path, 'r') as f:
        return f.read()
```

**Benefits:**

* 🧠 Auto-generates tool name, schema, and description
* 🧼 Keeps logic and documentation in one place
* 🏷️ Adds tags for organization

---

## 🛠️ How the Decorator Works

### `register_tool(...)`

A decorator that registers your function as an agent tool:

```python
def register_tool(tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    def decorator(func):
        metadata = get_tool_metadata(func, tool_name, description, parameters_override, terminal, tags)

        tools[metadata["tool_name"]] = {
            "description": metadata["description"],
            "parameters": metadata["parameters"],
            "function": metadata["function"],
            "terminal": metadata["terminal"],
            "tags": metadata["tags"]
        }

        # Optional: group tools by tags
        for tag in metadata["tags"]:
            tools_by_tag.setdefault(tag, []).append(metadata["tool_name"])

        return func
    return decorator
```

---

### 🔍 `get_tool_metadata(...)`

This helper:

* Pulls function name and docstring
* Parses parameters with `inspect` and `typing.get_type_hints()`
* Converts Python types into JSON Schema
* Flags required arguments
* Ignores special parameters like `action_context`

```python
def get_tool_metadata(func, tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    tool_name = tool_name or func.__name__
    description = description or (func.__doc__.strip() if func.__doc__ else "No description provided.")

    if parameters_override is None:
        # Build schema from function signature
        ...
    else:
        args_schema = parameters_override

    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or []
    }
```

---

## 🚀 Why Use This Pattern?

| Benefit                       | Description                                               |
| ----------------------------- | --------------------------------------------------------- |
| 📘 **Single Source of Truth** | Function = code + doc + schema                            |
| 🔄 **Automatic Updates**      | Changing the function auto-updates the registration       |
| 🧩 **Better Modularity**      | Tools stay self-contained and composable                  |
| 🏷️ **Categorization**        | Tags allow grouping by domain (e.g., `"file_operations"`) |

---

## 🌟 Real Example of Power

Changing this:

```python
def read_file(file_path: str):
```

To this:

```python
def read_file(file_path: str, encoding: str = "utf-8"):
```

Automatically:

* Updates the parameter schema
* Makes `encoding` optional (since it has a default)
* Updates docs if changed

No extra lines to edit!






### 🧠 **Decorator as Metadata Collector**

The **tool decorator** (`@register_tool`) acts like a smart indexer or librarian:

| Aspect                  | What It Does                                                 |
| ----------------------- | ------------------------------------------------------------ |
| **Collects metadata**   | Grabs the tool name, docstring, argument names/types, etc.   |
| **Stores in registry**  | Adds the tool to a centralized dictionary (`tools`)          |
| **Standardizes schema** | Converts parameters into JSON schema format                  |
| **Adds tags**           | Optionally tags tools by category (like `"file_operations"`) |

So yes — the decorator becomes a central *interface layer* between raw Python functions and agent reasoning.

---

### 🧭 How It Relates to AgentLanguage

| Role                | Decorator (`@register_tool`)               | AgentLanguage Class                              |
| ------------------- | ------------------------------------------ | ------------------------------------------------ |
| **What it manages** | Tool metadata and registration             | Prompt construction and response parsing         |
| **Acts on**         | Individual tool functions                  | The entire interaction with the LLM              |
| **Purpose**         | Makes tools introspectable and agent-ready | Connects goals, memory, tools into an LLM prompt |
| **Example analogy** | Like indexing books in a library           | Like writing and reading the book summaries      |

---

### 🔧 Summary

* The **tool decorator** prepares and organizes the tools.
* The **AgentLanguage** class builds a conversation interface *using* those tools.
* Together, they create a robust and dynamic agent system without manual duplication or brittle code.






### 🧠 Old Way (Manual Approach)

In the old approach, tools were **defined and registered separately**, like this:

```python
def get_weather(location: str) -> str:
    ...

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Gets weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"}
                },
                "required": ["location"]
            }
        }
    }
]
```

⚠️ The problem:

* Change the function name? You have to update it in the registry.
* Add a new parameter? Update the schema manually.
* Change the description? Update both docstring and JSON.

**Prone to error. Tedious. Brittle.**

---

### ✅ Decorator Approach (Automatic + DRY)

With decorators, you do **everything in one place**:

```python
@register_tool(name="get_weather", description="Get the weather for a given city.")
def get_weather(location: str):
    ...
```

And if you change:

* The function name: It’s used directly.
* The description: You just update the docstring or `description` field.
* The parameters: The decorator introspects the function signature and updates automatically.

You also get:

* ✅ Less duplication
* ✅ Safer updates
* ✅ Easier testing and documentation

---

### 🔧 `parameters_override`

Even when you **need to customize** beyond what Python can infer, you can override just the piece you care about:

```python
@register_tool(
    name="get_weather",
    description="Get the weather for a location",
    parameters_override={
        "location": {"type": "string", "description": "City and country, e.g., 'London, UK'"}
    }
)
def get_weather(location: str):
    ...
```

So you're still **not duplicating the entire schema**, just enhancing parts of it.

---

### 🧩 Final Thought

This all fits beautifully with the **AgentFunctionCallingActionLanguage** design — the LLM sees the clean, well-organized tools **automatically**, and you maintain just the function.




Let’s walk through the **example step-by-step** and explore what’s happening under the hood with decorators and tagging. 🛠️

---

## 🧠 The Decorator’s Purpose

You're using `@register_tool(...)` as a decorator to:

1. Automatically **register the tool**
2. Attach useful **metadata**, such as `tags`, `name`, `description`, etc.
3. Store that tool in two **global registries**:

   * `tools`: lookup by name
   * `tools_by_tag`: lookup by purpose/function

---

## 🔍 Code Breakdown

### 1. Decorating and Registering

```python
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    ...
```

When Python hits this line:

* It wraps `read_file` with the `register_tool` function.
* It infers the name as `"read_file"` (unless overridden).
* It stores it in `tools` and updates `tools_by_tag["file_operations"]` to include `"read_file"`.

Same goes for:

```python
@register_tool(tags=["file_operations", "write"])
def write_file(file_path: str, content: str) -> None:
```

Now `"write_file"` appears under both `file_operations` and `write`.

---

### 2. Global Registries (Automatically Built)

Assuming your decorator is implemented like this:

```python
tools = {}
tools_by_tag = {}

def register_tool(tags=None):
    def wrapper(func):
        name = func.__name__
        tools[name] = func

        for tag in tags or []:
            tools_by_tag.setdefault(tag, []).append(name)

        return func
    return wrapper
```

You end up with:

```python
tools = {
    "read_file": <function read_file>,
    "write_file": <function write_file>,
    "query_database": <function query_database>
}

tools_by_tag = {
    "file_operations": ["read_file", "write_file"],
    "write": ["write_file"],
    "database": ["query_database"],
    "read": ["query_database"]
}
```

---

## ✅ Why This Is Awesome

| Feature               | Benefit                                                                                 |
| --------------------- | --------------------------------------------------------------------------------------- |
| **Tags**              | Lets you group/search/filter tools by purpose                                           |
| **Auto-registration** | Reduces manual boilerplate                                                              |
| **Scalable registry** | Supports hundreds of tools without getting messy                                        |
| **LLM context**       | You can feed only specific categories of tools into the LLM (e.g., `"read"` tools only) |



This is an excellent and practical use of the tool tagging system you saw earlier. Let’s break it down step by step and see **why this pattern is so powerful** for scalable and modular AI agent development.

---

## 🧱 Core Concept: **Filtered Action Registries**

Instead of manually adding each action one-by-one like:

```python
action_registry.register(Action(...))
```

You're saying:

> "Give me all the tools with the tag `'file_operations'` and turn them into actions."

So, **each agent gets only the tools it actually needs**.

---

## 🔁 What’s Happening Under the Hood?

Let’s assume you’ve got:

```python
@register_tool(tags=["file_operations"])
def read_file(...): ...

@register_tool(tags=["database"])
def query_database(...): ...
```

Then, `ActionRegistry(tags=["file_operations"])` likely does something like this:

```python
def __init__(self, tags: List[str] = None):
    self.actions = {}

    if tags:
        for tag in tags:
            for tool_name in tools_by_tag.get(tag, []):
                func = tools[tool_name]
                # Wrap as Action
                action = Action(
                    name=tool_name,
                    function=func,
                    description=func.__doc__ or "",
                    parameters=... # inferred or passed
                )
                self.register(action)
```

---

## ✅ Benefits

| Benefit                       | Description                                                                        |
| ----------------------------- | ---------------------------------------------------------------------------------- |
| 🔌 **Pluggable agents**       | You can mix and match tools by domain (file, database, network, etc.)              |
| 🧼 **Minimal tool exposure**  | The LLM only sees tools it needs for that job—reduces confusion, improves accuracy |
| 🧪 **Easy testing**           | You can simulate and test agents in isolation                                      |
| 🔧 **Better maintainability** | No duplication across agents—just reuse tagged tools                               |

---

## 🧠 Agent Specialization Example

```python
def create_file_processing_agent():
    action_registry = ActionRegistry(tags=["file_operations"])

    return Agent(
        goals=[Goal(1, "File Processing", "Work with project files")],
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=Environment()
    )
```

You get:

* Only tools like `read_file`, `write_file`, etc.
* Cleaner LLM context
* Higher reliability in tool selection



This final section ties everything together by demonstrating **how tagging + ActionRegistry enables rapid agent specialization**. Let’s walk through what’s happening:

---

## 🧠 Summary: Creating Specialized Agents with Tags

Instead of manually defining which tools each agent can use, we **just declare a tag**, and the system auto-builds the action list behind the scenes. That means:

* ✅ **Less manual configuration**
* ✅ **Clear separation of concerns**
* ✅ **Easier to reason about agent behavior**

---

### 📌 Example 1: Read-Only Agent

```python
read_only_agent = Agent(
    goals=[Goal(1, "Read Only", "Read but don't modify data")],
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["read"]),
    generate_response=generate_response,
    environment=Environment()
)
```

**Behavior:**

* Can use only tools that were tagged with `"read"` — like `read_file`, `query_database`, etc.
* Won’t see or call `write_file` or `delete_file` even if they exist
* This is a *safety mechanism* to enforce read-only behavior at the tool-access level

---

### 📌 Example 2: File Operations Agent

```python
file_agent = Agent(
    goals=[Goal(1, "File Handler", "Manage file operations")],
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["file_operations"]),
    generate_response=generate_response,
    environment=Environment()
)
```

**Behavior:**

* Gets access to `read_file`, `write_file`, etc.
* This agent is empowered to **read, write, and possibly edit files**, but nothing else (e.g., no database access)

---

## 💡 Why This Design Is Powerful

| Feature                     | Benefit                                                                   |
| --------------------------- | ------------------------------------------------------------------------- |
| 🎯 Focused tool access      | Prevents agents from misusing tools outside their scope                   |
| 🧱 Easy composability       | Agents can be spun up just by tagging new tools and reusing the framework |
| 🔐 Safer execution          | Helps ensure only appropriate tools are available to each agent           |
| 📦 Better code organization | Tags group tools semantically (e.g., "read", "file\_operations", "db")    |


